### doc_entity
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E ENTIDADES NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

AS ENTIDADES A SEREM TRABALHADAS SÃO:
* TÓPICOS
* PAÍSES
* PESSOAS

In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import re
import pymysql
#import psycopg2
import getpass

from sshtunnel import SSHTunnelForwarder
import time

import codecs
from langdetect import detect
from IPython.display import clear_output
import csv

In [2]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
define pastas de trabalho
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

path = "C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/"
#path = "C:/Users/marcelo.ribeiro/Documents/temp/" #TESTE!
#outputs = "C:/Users/marcelo.ribeiro/Documents/outputs/"
#path = "/home/rsouza/Documentos/text-learning-tools/textfiles-corrected-regrouped/"

files = [f for f in os.listdir(path)]
fullpath_list = []
fullpath = ''
for file in files:
    fullpath = path+file
    fullpath_list.append(fullpath)

In [3]:
files[0:4]

['AAS_mre_ag_1973.11.20_doc_I-1.txt',
 'AAS_mre_ag_1973.11.20_doc_I-10.txt',
 'AAS_mre_ag_1973.11.20_doc_I-100.txt',
 'AAS_mre_ag_1973.11.20_doc_I-101.txt']

In [ ]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
alimenta a base mysql

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
temp = 0

doc_class = []
lang_class = 0
count_doc = 0
not_readable = []
percentil = int(len(fullpath_list)/100)

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        #ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        #ssh_private_key_password = input('Entre com a senha da chave privada'),
                        ssh_username="marcelobribeiro",
                        #ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           #user='rsouza',
                           passwd=pass_mysql,
                           #passwd=input('Entre com a senha do usuário no banco de dados'),
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE marcelo_draft")
    cur.execute('''CREATE TABLE IF NOT EXISTS marcelo_draft 
               (id VARCHAR(50), body LONGTEXT, year VARCHAR(4), month VARCHAR(2), url VARCHAR(200), main_language VARCHAR(10), temp VARCHAR(2)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')
    cur.execute("DELETE FROM marcelo_draft WHERE temp = 0")
    
    for txt in fullpath_list[0:200]:
        
        ### mede percentual de conclusão da tarefa ###
        count_doc += 1
        if count_doc % percentil == 0: print(int(count_doc/percentil),'% done')
        if count_doc % (percentil-1) == 0: clear_output()
        
                
        with open(txt, 'r', encoding='utf-8') as f: #obs: deveria usar encoding utf-8. Ignorando erros
            txt_body = f.read()

            
            ### insere dados na base mysql ###
            query = "INSERT INTO marcelo_draft VALUES (%s, %s, %s, %s, %s, %s, %s)"
            cur.execute(query, (txt_id, txt_body, txt_year, txt_month, url, lang_class, temp))

    cur.execute("SELECT * FROM marcelo_draft");
    rows = cur.fetchall()
    
    print(rows)